# Yolo Object detection training
* Download normalizaed coco dataset here: https://drive.google.com/file/d/14xELdFQwyfnbFUvdnoCu5WpXCso4xc2-/view?usp=sharing
* After downloading, upload to this colab file!
* Or you can use the code below to download using the file id: 14xELdFQwyfnbFUvdnoCu5WpXCso4xc2-

In [ ]:
!rm -rf /content/*

In [ ]:
# https://stackoverflow.com/a/62568654
!gdown --id 14xELdFQwyfnbFUvdnoCu5WpXCso4xc2-

In [ ]:
# Download data
!wget http://images.cocodataset.org/zips/train2014.zip
!mkdir train
!unzip /content/train2014.zip -d /content/train/
!rm -rf /content/train2014.zip

!wget http://images.cocodataset.org/zips/val2014.zip
!mkdir val
!unzip /content/val2014.zip -d /content/val/
!rm -rf /content/val2014.zip

In [ ]:
# clone library
!git clone https://github.com/rxng8/YOLO-Object-Detection-Algorithm

In [ ]:
# Move file
import shutil
import os
source = '/content/YOLO-Object-Detection-Algorithm'
dest = '/content/'
files = os.listdir(source)
for f in files:
  shutil.move(os.path.join(source, f), os.path.join(dest, f))
!rm -rf '/content/YOLO-Object-Detection-Algorithm'

In [ ]:
# Install library
# !pip uninstall tensorflow
# !pip install tensorflow-gpu==2.6.0
!pip install matplotlib
!pip install opencv-python
!pip install tqdm
# !pip install keras==2.6.0

In [ ]:
import collections
import sys
import os
from typing import List, Dict, Tuple
import json
import csv
import pickle

import tensorflow as tf
print(f"Tensorflow version: {tf.__version__}")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.experimental.list_physical_devices('CPU'))
print(tf.config.experimental.list_physical_devices('GPU'))
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow.keras import layers
import numpy as np
import pandas as pd
from PIL import ImageDraw, ImageFont, Image
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

import tqdm

from yolo.const import *
from yolo.utils import draw_boxes, dynamic_iou, iou, show_img, preprocess_image, show_img_with_bbox
from yolo.model import SimpleModel, \
  SimpleModel2, SimpleYolo, SimpleYolo2, testSimpleYolo, \
  SimpleYolo3, SimpleYolo4, make_yolov3_model
from yolo.loss import yolo_loss, simple_mse_loss, yolo_loss_2, yolo_loss_3

dataset_root = "/content/"
train_image_folder = os.path.join(dataset_root, "train/train2014")
test_image_folder = os.path.join(dataset_root, "val/val2014")

In [ ]:

data_pile_path = "/content/dump.npy"
with open(data_pile_path, "rb") as f:
    [classes, \
    class_to_id, \
    id_to_class, \
    train_data, \
    id_to_train_image_metadata, \
    test_data, \
    id_to_test_image_metadata] = np.load(f, allow_pickle=True)

    n_class = len(classes)
    train_data_size = len(list(train_data))
    test_data_size = len(list(test_data))

In [ ]:

def train_gen():
  # [(image_id, [{bbox: list, category: str}]]
  item_list = list(train_data.items())
  pointer = 0
  while True:
    if pointer >= len(item_list):
      pointer = 0
    try:
      ### Perform generator here
      image_id, item_data = item_list[pointer]
      # print(id_to_train_image_metadata[image_id])
      
      image_name = id_to_train_image_metadata[image_id]["file_name"]
      image_path = os.path.join(train_image_folder, image_name)
      original_img = np.asarray(Image.open(image_path))
      original_width = id_to_train_image_metadata[image_id]["width"]
      original_height = id_to_train_image_metadata[image_id]["height"]
      preprocessed_img = preprocess_image(original_img, image_size=example_image_size)
      # show_img(preprocessed_img)

      label = np.zeros(
        shape=(n_cell_y, n_cell_x, n_class + 5),
        dtype=float
      )

      for box_data in item_data:
        original_x, original_y, original_box_width, original_box_height = box_data["bbox"]
        _class = box_data["category"]
        _class_id = class_to_id[_class]

        # Compute regarding to the currnet image. All positions range [0, 1]
        x = float(original_x) / original_width
        y = float(original_y) / original_height
        box_width = float(original_box_width) / original_width
        box_height = float(original_box_height) / original_height
        center_x = x + box_width / 2.0
        center_y = y + box_height / 2.0
        # print(f"x: {x}, y: {y}, box_width: {box_width}, box_height: {box_height}, center_x: {center_x}, center_y: {center_y}")

        # compute the coordinates center with regard to the current cell
        xth_cell = int(center_x * n_cell_x)
        yth_cell = int(center_y * n_cell_y)
        cell_center_x = center_x * n_cell_x - xth_cell
        cell_center_y = center_y * n_cell_y - yth_cell
        cell_box_width = box_width * n_cell_x
        cell_box_height = box_height * n_cell_y

        if label[yth_cell, xth_cell, n_class + 4] == 0:
          label[yth_cell, xth_cell, _class_id] = 1.0
          label[yth_cell, xth_cell, n_class: n_class + 4] = cell_center_x, cell_center_y, cell_box_width, cell_box_height
          label[yth_cell, xth_cell, n_class + 4] = 1.0
        # boxed = draw_boxes(preprocessed_img, [[x, y, box_width, box_height]])
        # print(f"class: {_class}")
        # show_img(boxed)

      yield {
        "input": preprocessed_img,
        "output": tf.convert_to_tensor(label)
      }

      ### End of generator performance
      pointer += 1
    except:
      pointer += 1
      continue

def test_gen():
  # [(image_id, [{bbox: list, category: str}]]
  item_list = list(test_data.items())
  pointer = 0
  while True:
    if pointer >= len(item_list):
      pointer = 0
    try:
      ### Perform generator here
      image_id, item_data = item_list[pointer]
      image_name = id_to_test_image_metadata[image_id]["file_name"]
      image_path = os.path.join(test_image_folder, image_name)
      original_img = np.asarray(Image.open(image_path))
      original_width = id_to_test_image_metadata[image_id]["width"]
      original_height = id_to_test_image_metadata[image_id]["height"]
      preprocessed_img = preprocess_image(original_img, image_size=example_image_size)
      label = np.zeros(
        shape=(n_cell_y, n_cell_x, n_class + 5),
        dtype=float
      )
      for box_data in item_data:
        original_x, original_y, original_box_width, original_box_height = box_data["bbox"]
        _class = box_data["category"]
        _class_id = class_to_id[_class]
        # Compute regarding to the currnet image. All positions range [0, 1]
        x = float(original_x) / original_width
        y = float(original_y) / original_height
        box_width = float(original_box_width) / original_width
        box_height = float(original_box_height) / original_height
        center_x = x + box_width / 2.0
        center_y = y + box_height / 2.0
        # compute the coordinates center with regard to the current cell
        xth_cell = int(center_x * n_cell_x)
        yth_cell = int(center_y * n_cell_y)
        cell_center_x = center_x * n_cell_x - xth_cell
        cell_center_y = center_y * n_cell_y - yth_cell
        cell_box_width = box_width * n_cell_x
        cell_box_height = box_height * n_cell_y
        if label[yth_cell, xth_cell, n_class + 4] == 0:
          label[yth_cell, xth_cell, _class_id] = 1.0
          label[yth_cell, xth_cell, n_class: n_class + 4] = cell_center_x, cell_center_y, cell_box_width, cell_box_height
          label[yth_cell, xth_cell, n_class + 4] = 1.0
      yield {
        "input": preprocessed_img,
        "output": tf.convert_to_tensor(label)
      }
      ### End of generator performance
      pointer += 1
    except:
      pointer += 1
      continue

In [ ]:
train_dataset = tf.data.Dataset.from_generator(train_gen, output_signature={
  "input": tf.TensorSpec(shape=(*example_image_size, n_channels), dtype=tf.float32),
  "output": tf.TensorSpec(shape=(n_cell_y, n_cell_x, n_class + 5), dtype=tf.float32)
})
print(train_dataset.element_spec)
train_batch_dataset = train_dataset.batch(batch_size=BATCH_SIZE)
train_batch_iter = iter(train_batch_dataset)

test_dataset = tf.data.Dataset.from_generator(test_gen, output_signature={
  "input": tf.TensorSpec(shape=(*example_image_size, n_channels), dtype=tf.float32),
  "output": tf.TensorSpec(shape=(n_cell_y, n_cell_x, n_class + 5), dtype=tf.float32)
})
test_dataset_iter = iter(test_dataset)
test_batch_dataset = train_dataset.batch(batch_size=BATCH_SIZE)
test_batch_iter = iter(test_batch_dataset)

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

In [ ]:
model: tf.keras.Model = make_yolov3_model()
model.summary()
with tf.device("/CPU:0"):
  test_logits = tf.random.normal((BATCH_SIZE, *example_image_size, n_channels), mean=0.5, stddev=0.3)
  test_pred = model(test_logits, training=False)
  print(test_pred.shape)

In [ ]:
def train_step(batch_x, batch_label, model, loss_function, optimizer, debug=False):
  with tf.device("/GPU:0"):
    with tf.GradientTape() as tape:
      logits = model(batch_x, training=True)
      loss, [loss_xy, loss_wh, loss_conf, loss_class] = loss_function(batch_label, logits)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
  if debug:
    return logits, loss, [loss_xy, loss_wh, loss_conf, loss_class]
  return loss, [loss_xy, loss_wh, loss_conf, loss_class]

def train(model, 
        training_batch_iter, 
        test_batch_iter, 
        optimizer, 
        loss_function,
        epochs=1, 
        steps_per_epoch=20, 
        valid_step=5,
        history_path=None,
        weights_path=None):
  
  if not os.path.exists(training_history_path):
    epochs_val_loss = np.array([])
    epochs_loss = np.array([])
    history = [epochs_loss, epochs_val_loss]
  else:
    with open(training_history_path, "rb") as f:
      history = np.load(f, allow_pickle=True)

  epochs_loss, epochs_val_loss = history
  epochs_loss = epochs_loss.tolist()
  epochs_val_loss = epochs_val_loss.tolist()

  if os.path.exists(model_weights_path + ".index"):
    try:
      model.load_weights(model_weights_path)
      print("Model weights loaded!")
    except:
      print("Cannot load weights")

  # https://philipplies.medium.com/progress-bar-and-status-logging-in-python-with-tqdm-35ce29b908f5
  # outer_tqdm = tqdm(total=epochs, desc='Epoch', position=0)
  loss_logging = tqdm.tqdm(total=0, bar_format='{desc}', position=1)
  
  for epoch in range(epochs):
    losses = []
    val_losses = []

    # https://towardsdatascience.com/training-models-with-a-progress-a-bar-2b664de3e13e
    # https://www.geeksforgeeks.org/python-how-to-make-a-terminal-progress-bar-using-tqdm/
    with tqdm.tqdm(total=steps_per_epoch, desc=f"Epoch {epoch + 1}", position=0, ncols=100, ascii =".>") as inner_tqdm:
      with tf.device("/CPU:0"):
        for step_pointer in range(steps_per_epoch):
          batch = next(training_batch_iter)
          batch_x = batch["input"]
          batch_label = batch["output"]
          loss, [loss_xy, loss_wh, loss_conf, loss_class] = train_step(
            batch_x, batch_label, 
            model, loss_function, optimizer)

          # Log?
          desc = f"Epoch {epoch + 1} - Step {step_pointer + 1} - Loss: {loss}"
          # loss_logging.set_description_str(desc)
          # print()

          losses.append((loss, [loss_xy, loss_wh, loss_conf, loss_class]))

          if (step_pointer + 1) % valid_step == 0:
            # desc = f"Training loss (for one batch) at step {step_pointer + 1}: {float(loss)}"
            # print(desc)
            # loss_logging.set_description_str(desc)

            # perform validation
            val_batch = next(test_batch_iter)
            logits = model(val_batch["input"], training=False)
            val_loss, [val_loss_xy, val_loss_wh, val_loss_conf, val_loss_class] = loss_function(val_batch["output"], logits)
            val_losses.append((val_loss, [val_loss_xy, val_loss_wh, val_loss_conf, val_loss_class]))
            # print(f"Validation loss: {val_loss}\n-----------------")

          inner_tqdm.set_postfix_str(f"Loss: {loss}")
          inner_tqdm.update(1)

    epochs_loss.append(losses)
    epochs_val_loss.append(val_losses)

    # Save history and model
    if history_path != None:
      np.save(history_path, [epochs_loss, epochs_val_loss])
    
    if weights_path != None:
      model.save_weights(weights_path)

    # outer_tqdm.update(1)

  # return history
  return [epochs_loss, epochs_val_loss]


In [ ]:
training_history_path = "/content/training_history/history9.npy"
model_weights_path = "/content/weights/checkpoint10"

history = train(
  model,
  train_batch_iter,
  test_batch_iter,
  optimizer,
  yolo_loss_3,
  epochs=1,
  steps_per_epoch=500, # 82783 // 4
  history_path=training_history_path,
  weights_path=model_weights_path
)